# Интерфейсы как соглашения

До сих пор мы рассматривали только конкретные типы данных. Мы всегда знаем, что из себя представляет конкретный тип данных и что с ним можно делать. В Go есть и другой тип данных, называемый **интерфейсным типом**. Интерфейсы по-другому называют **абстрактными типами**. Интерфейсный тип не раскрывает внутреннюю структуру своих значений или множество операций, которые поддерживает, а лишь предоставляет методы. Когда у нас есть значение интерфейсного типа мы не знаем, что он из себя представляет; мы только знаем что с ним можно делать, а точнее, какое поведение предоставляется его методами.

Интерфейсы можно рассматривать как соглашения. Если некий тип удовлетворяет интерфейсу, то значения такого типа можно использовать ввезде, где используется этот интерфейс. Рассмотрим например функции Printf и Sprintf из пакета fmt. Эти функции являются лишь обёртками над более обобщённой функцией Fprintf. Fprintf имеет следующую сигнатуру:

In [ ]:
func Fprintf(w io.Writer, format string, a ...interface{}) (n int, err error)

Первым аргументом этой функции является интерфейс Writer. Этот интерфейс имеет следующий вид:

In [ ]:
type Writer interface {
    Write(p []byte) (n int, err error)
}

Любой тип, который определяет метод Write именно с такой сигнатурой, которая определена в этом интерфейсе, удовлетворяет интерфейсу Writer, а значит, значения такого типа можно передать функции Fprintf. Именно так и работают функции Printf и Sprintf: Printf передаёт значение типа \*os.File, а Sprintf - \*bytes.Buffer. Оба этих типа удовлетворяют интерфейсу Writer(почему необходимо использовать именно указатели мы узнаем позже). Можно придумать и свой тип, который удовлетворяет Writer'у:

In [1]:
import "fmt"

type ByteCounter int

func (c *ByteCounter) Write(p []byte) (int, error) {
    *c += ByteCounter(len(p))
    return len(p), nil
}

var c ByteCounter
fmt.Fprintf(&c, "Brand New %s", "World")
fmt.Sprint(c)

15


Есть также и другой очень распространённый интерфейс, который позволяет выводить свои типы данных с использованием функций пакета fmt. Это интерфейс Stringer:

In [ ]:
type Stringer interface {
    String() string
}

Рассмотрим пример:

In [2]:
type Color uint32

func (c *Color) String() string {
    return fmt.Sprintf("R:%X, G:%X, B:%X, A:%X", (*c >> 24) & 0xFF, (*c >> 16) & 0xFF, (*c >> 8) & 0xFF, *c & 0xFF)
}

var c Color
c = 0xAABBCCFF
fmt.Sprint(&c)

R:AA, G:BB, B:CC, A:FF


# Интерфейсный тип

Интерфейсный тип определяет множество методов, который должен определить конкретный тип, чтобы его можно было рассматривать как экземпляр этого интрефейса.

**io.Writer** является наиболее используемым интерфейсом, т.к. он предоставляет абстракцию всех типов, в которые можно что-то записать, как например файлы, буферы в памяти, сетевые соединения и т.д. Пакет io определяет ещё и другие полезные интерфейсы:**Reader** представляет тип, из которого можно что-то вычитать, **Closer** - абстракция для всего, что можно закрыть:

In [ ]:
type Reader interface {
    Read(p []byte) (n int, err error)
}

type Writer interface {
    Write(p []byte) (n int, err error)
}

type Closer interface {
    Close() error
}

Интерфейсы также поддерживают **встраивание**(embedding): новые интерфейсы можно определить на основе существующих. Для интерфейса встраивание означает, что он получает все методы, которые были объявлены в встраиваемом интерфейсе. Пример:

In [ ]:
type ReadWriter interface {
    Reader
    Writer
}

Этот интерфейс эквивалентен следующему:

In [ ]:
type ReadWriter interface {
    Read(p []byte) (n int, err error)
    Write(p []byte) (n int, err error)
}

# Соответствие интерфейсу

Тип **соответствует**(или удовлетворяет(satisfies)) интерфейсу, если он определяет все методы, которые были объявлены в этом интерфейсе. Например, \*bytes.Buffer соответствует интерфейсам Reader, Writer и ReadWriter, но не интерфейсу Closer, т.к. в нём нет метода Close. Разработчики на Go говорят, что конкретный тип является("is") интерфейсным типом, имея в виду, что он удовлетворяет интерфейсу. Например, можно сказать, что \*bytes.Buffer является io.Writer'ом.

Правило **присваиваемости** для интерфейсов звучит очень просто: выражение можно присвоить интерфейсу, только если его тип соответствует интерфейсу:

In [3]:
import (
    "io"
    "os"
    "bytes"
    "time"
)

var w io.Writer
w = os.Stdout         // Ошибки нет: *os.File определяет метод Write.
w = new(bytes.Buffer) // Ошибки нет: *bytes.Buffer определяет метод Write.
w = time.Second       // Есть ошибка: time.Second не определяет метод Write.

11:5: cannot use time.Second (constant 1000000000 of type time.Duration) as io.Writer value in assignment: missing method Write


Ещё пример:

In [4]:
import (
    "io"
    "os"
    "bytes"
)

var rwc io.ReadWriteCloser
rwc = os.Stdout         // Ошибки нет: *os.File определяет метод Write.
rwc = new(bytes.Buffer) // Есть ошибка: bytes.Buffer не определяет метод Close.

9:7: cannot use new(bytes.Buffer) (value of type *bytes.Buffer) as io.ReadWriteCloser value in assignment: missing method Close


Это правило применимо даже когда правая часть сама по себе является интерфейсом:

In [5]:
import "io"

var w io.Writer
var rwc io.ReadWriteCloser
w = rwc  // Ошибки нет
rwc = w  // Есть ошибка

6:7: cannot use w (variable of type io.Writer) as io.ReadWriteCloser value in assignment: missing method Close


## Замечание по соответствию интерфейсу

В Go есть тонкая особенность, связанная с определением того, входит ли метод в тип. Когда мы рассматривали методы в главе 06.01 мы говорили, что конкретный тип T может объявить некоторые методы с получателем значением типа T, а некоторые с получателем указателем типа \*T и для аргумента типа T мы могли вызвать метод с получателем \*T, если этот аргумент являлся **переменной**: компилятор неявно извлекал адрес. Но это является лишь **синтаксическим сахаром**: значения типа T **не владеют** всеми методами, которыми владеют значения типа \*T, а следовательно могут соответствовать меньшему количеству интерфейсов. Давайте рассмотрим пример, чтобы всё стало яснее:

In [6]:
import "fmt"

type Foo int

func (f *Foo) String() string {
    return fmt.Sprint("%d", *f)
}

Методом String владеет только тип \*Foo, а следовательно только \*Foo удовлетворяет интерфейсу Stringer:

In [7]:
var f Foo
var s fmt.Stringer = &f // Ошибки нет, т.к. *Foo владеет методом String.
var t fmt.Stringer = f  // Есть ошибка, т.к. Foo не владеет методом String.

3:22: cannot use f (variable of type github.com/yunabe/lgo/sess7b2274696d65223a313533343538353034373335343439353837327d/exec6.Foo) as fmt.Stringer value in variable declaration: missing method String


# Пустой интерфейс

Интерфейс io.Writer утверждает, что конкретные типы, которые соответствуют этому интерфейсу, поддерживают запись. О чём может тогда утверждать **пустой интерфейс interface{}**, который вообще не содержит методов? Верно: ничего. Т.к. пустой интерфейс не налагает на конкретный тип каких-либо тревований, **любой тип** соответствует этому интерфейсу:

In [8]:
var any interface{}
any = 42
any = true
any = map[string]int{"one":1}

От того что мы создали переменую типа пустого интерфейса и присвоили ей значение мы много не выиграли, т.к. у пустого интерфейса нет методов, а следовательно не можем ничего с ним сделать. Нам необходим способ вернуться обратно к тому значению, которые было присвоено интерфейсу. Мы рассмотрим как это можно сделать, используя **type assertion** в последующих главах.